In [ ]:
%pip install fastai duckduckgo_search

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *
from fastai.vision.all import *
from time import sleep

In [ ]:
path = Path('data.ignore/bird_or_not')

## Dowload images

In [ ]:
def search_images(term, max_images=200):
  return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [ ]:
urls = search_images('bird photos', max_images=1)
urls[0]

In [ ]:
dest = 'data.ignore/bird.jpg'
download_url(urls[0], dest, show_progress=False)

im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
download_url(search_images('forest photos', max_images=1)[0], 'data.ignore/forest.jpg', show_progress=False)
Image.open('data.ignore/forest.jpg').to_thumb(256,256)

In [ ]:
searches = 'forest','bird'

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} sun photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} shade photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

## Process images

In [ ]:
# Dataloaders are what Pytorch uses to iterate your data
dataloaders = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dataloaders.show_batch(max_n=6)

In [ ]:
learner = vision_learner(dataloaders, resnet18, metrics=error_rate)
learner.fine_tune(3)

In [ ]:
is_bird, _, probs = learner.predict(PILImage.create('data.ignore/bird.jpg'))

print(is_bird)
print(f"{probs[0]:.4f}")